# TODO notebook inteiro, por ora é só cópia do Notebook do FakeRecogna!! Combinar datasets e treinar

# Treinamento e validação de Modelos - Dataset FakeRecogna
Treinamento de um modelo usando como base o segundo dataset estudado: https://github.com/Gabriel-Lino-Garcia/FakeRecogna 

Após verificar que o primeiro modelo obteve uma performace ruim ao ser testado com outro dataset, fez-se a hipótese de que isto pode ter ocorrido devido à baixa variabilidade de notícias no primeiro dataset utilizado. Optou-se então por tentar treinar um modelo TF-IDF no segundo dataset, que possui maior variedade de notícias, para validar se isto resultaria em um modelo mais geral.

Este modelo, além de ser validado no próprio dataset que o gerou, também é validado com o primeiro dataset utilizado no projeto.

In [2]:
# Se necessário
%pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 435 kB 21.9 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sagemaker
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report
from scipy.sparse import csr_matrix, hstack
from python_scripts.save_load import load_df_from_bucket, save_df_to_s3_bucket, save_to_s3_bucket_as_libsvm, BUCKET_MODEL
from python_scripts.modelling import create_train_validation_test_sets, setup_model, make_prediction

## Carregamento de dados e processamento inicial do dataset
Como foi verificado no notebook anterior, sabe-se que há uma linha de nulos que deve ser eliminada do  dataset. Além disso, os labels de notícia falsa/verdadeira são corrigidos para o padrão utilizado no projeto (1 para Falsa, 0 para verdadeira)

In [3]:
model_df = load_df_from_bucket('dados_processados_recogna.csv', tipo='processado')
model_df.head()

,Categoria,lemmas_str,fake
0,entretenimento,apagão vaticano papar presar acusação tráfico ...,1
1,saúde,governar equador anunciar preparar cova coleti...,0
2,saúde,companhia air france operar voar direto pequim...,0
3,saúde,marfrig global foods retomar vender carnar bov...,0
4,entretenimento,assunto voltar compartilhar rede social julho ...,1


## Modelo 1: TF-IDF baseado no texto da notícia
Vetorização TF-IDF é aplicada somente à coluna de texto da notícia. Demais colunas de dados não são consideradas.

In [4]:
train_1, test_1, validate_1 = create_train_validation_test_sets(model_df, 
                                                                stratify_col='fake',
                                                                test_size=0.2, random_state=42)

### Processamento adicional

Um vetorizador TFIDF é utilizado para converter os dados textuais em colunas do DataFrame.

In [5]:
# Criar vetorizador TFIDF e ajustar aos dados de treinamento
tfidf = TfidfVectorizer(lowercase=False, ngram_range = (1,2))
tfidf.fit(train_1['lemmas_str'])

TfidfVectorizer(lowercase=False, ngram_range=(1, 2))

In [6]:
def create_x_y_1(base_df, tfidf, target_col='fake', lemma_col = 'lemmas_str'):
    tfidf_res = tfidf.transform(base_df[lemma_col])
    return tfidf_res, base_df[target_col]

x_train_1, y_train_1 = create_x_y_1(train_1, tfidf)
x_validate_1, y_validate_1 = create_x_y_1(validate_1, tfidf)
x_test_1, y_test_1 = create_x_y_1(test_1, tfidf)

### Upload de dados para o S3

In [7]:
file_name_tuples = [(x_train_1, y_train_1, 'train'), 
                   (x_test_1, y_test_1, 'test'), 
                   (x_validate_1, y_validate_1, 'validate')]

for x, y, prefix in file_name_tuples:
    save_to_s3_bucket_as_libsvm(x, y, prefix=prefix, filename='model_1_2.libsvm', tipo='modelo')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazon

### Treinar modelo

In [9]:
xgb_model_1, data_channels_1 = setup_model(base_image='xgboost', model_name='model_1_2', instance_count=4, 
                                           instance_type='ml.m4.xlarge')
xgb_model_1.fit(inputs=data_channels_1)

print('ready for hosting!')

2022-11-14 20:32:07 Starting - Starting the training job...ProfilerReport-1668457927: InProgress
...
2022-11-14 20:32:43 Starting - Preparing the instances for training...............
2022-11-14 20:35:22 Downloading - Downloading input data...
2022-11-14 20:36:02 Training - Training image download completed. Training in progress.[2022-11-14 20:35:59.983 ip-10-0-187-94.ec2.internal:8 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-11-14:20:36:00:INFO] Imported framework sagemaker_xgboost_container.training
[2022-11-14:20:36:00:INFO] Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
[2022-11-14:20:36:00:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2022-11-14:20:36:00:INFO] No GPUs detected (normal if no gpus installed)
[2022-11-14:20:36:00:INFO] Running XGBoost Sagemaker in algorithm mode
[2022-11-14:20:36:00:INFO] files path: /opt/ml/input/data/train
[2022-11-14:20:36:00:INFO]

In [10]:
xgb_predictor_1 = xgb_model_1.deploy(initial_instance_count=1,
                                     serializer=sagemaker.serializers.LibSVMSerializer(),
                                     instance_type='ml.m4.xlarge')

--------!

### Métricas do modelo

In [12]:
y_pred_1 = make_prediction(xgb_predictor_1, model_name='model_1_2')

In [13]:
print(classification_report(y_test_1, y_pred_1))

              precision    recall  f1-score   support

           0       0.93      0.85      0.89       595
           1       0.86      0.93      0.90       595

    accuracy                           0.89      1190
   macro avg       0.90      0.89      0.89      1190
weighted avg       0.90      0.89      0.89      1190



Verifica-se aqui um excelente desempenho mesmo com os dados separados para teste, como foi visto no notebook `model_fakebr.ipynb`para o dataset Fake.br-Corpus. Em seguida, será feita uma segunda validação utilizando este dataset.

### Salvar predição

In [ ]:
df_pred_alt = pd.DataFrame({'pred_1_2': y_pred_1})1
df_pred_alt.to_csv(f's3://{BUCKET_MODEL}/test/pred_1_2.csv', encoding='utf-8', index=False)

## Teste adicional - Dataset Fake.br-Corpus
Serão amostradas 800 notícias do Dataset Fake.br-Corpus para uma segunda validação do presente modelo.

### Carregar dataset e amostrar dados

In [17]:
alternative_df = load_df_from_bucket('dados_processados.csv', tipo='processado')
alternative_df.head()

,fake,text,avg_sent_len,avg_word_len,words_str,lemmas_str
0,0,A divisão do STF ao meio entre partidários e ...,10.504673,6.785107,divisão STF meio partidários independentes fic...,divisão STF meio partidário independente ficar...
1,1,"General manda recado para STF: ""Abaixaram as c...",10.866667,6.502610,General manda recado STF Abaixaram calças Cong...,general mandar recado STF abaixar calça congre...
2,1,O Nordeste acordou! Lula e o PT são enxotados:...,7.333333,6.565873,Nordeste acordou Lula PT enxotados Chega bande...,nordeste acordar Lula PT enxotar chegar bandei...
3,0,"Dois relatórios da Polícia Federal, com análi...",16.878788,7.286668,Dois relatórios Polícia Federal análises mater...,dois relatório Polícia Federal análise materia...
4,1,Coreia do Norte declara status de QUASE-GUERRA...,11.600000,6.531320,Coreia Norte declara status QUASE-GUERRA mobil...,Coreia Norte declarar status QUASE-GUERRA mobi...


In [18]:
alternative_df = alternative_df[['fake', 'lemmas_str']]
alternative_df.head()

,fake,lemmas_str
0,0,divisão STF meio partidário independente ficar...
1,1,general mandar recado STF abaixar calça congre...
2,1,nordeste acordar Lula PT enxotar chegar bandei...
3,0,dois relatório Polícia Federal análise materia...
4,1,Coreia Norte declarar status QUASE-GUERRA mobi...


In [22]:
sample_df = alternative_df.sample(n=800)

#### Predição

Aplicar TF-IDF para vetorizar, salvar dados no S3 e efetuar predição

In [23]:
x_test_alt, y_test_alt = create_x_y_1(sample_df, tfidf)
save_to_s3_bucket_as_libsvm(x_test_alt, y_test_alt, 
                            prefix='test', filename='model_2_1_alt.libsvm', tipo='modelo')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [24]:
y_pred_alt = make_prediction(xgb_predictor_1, model_name='model_2_1_alt')

## Métricas

In [25]:
print(classification_report(y_test_alt, y_pred_alt))

              precision    recall  f1-score   support

           0       0.64      0.94      0.77       381
           1       0.91      0.52      0.66       419

    accuracy                           0.72       800
   macro avg       0.78      0.73      0.71       800
weighted avg       0.78      0.72      0.71       800



Os resultados neste modelo foram bem mais razoáveis. Embora o modelo obtido não obtenha resultados ideais ao avaliar as notícias do outro dataset, são resultados mais coerentes.

Um modelo combinado, treinado baseado nos dois datasets, é apresentado no notebook `model_combinado.ipynb`.

#### Salvar predição

In [26]:
df_pred_alt = pd.DataFrame({'pred_2_alt': y_pred_alt})
df_pred_alt.to_csv(f's3://{BUCKET_MODEL}/test/pred_2_alt.csv', encoding='utf-8', index=False)

### Encerrar modelo
Após pegar métricas e dados de interesse, encerrar modelo para evitar cobranças.

In [27]:
xgb_predictor_1.delete_endpoint(delete_endpoint_config=True) 

In [50]:
df_pred_2 = pd.DataFrame({'pred_2': y_pred_2})
df_pred_2.to_csv(f's3://{BUCKET_MODEL}/test/pred_2.csv', encoding='utf-8', index=False)

In [76]:
dset_link = 'https://github.com/Gabriel-Lino-Garcia/FakeRecogna/raw/master/dataset/FakeRecogna.xlsx'
alternative_df = pd.read_excel(dset_link, engine='openpyxl')  # engine para xlsx
alternative_df.head()

,Titulo,Subtitulo,Noticia,Categoria,Data,Autor,URL,Classe
0,\n\nPapa Francisco foi preso sob acusação de t...,Boato – Ocorreu um apagão no Vaticano. O papa ...,apagão vaticano papar presar acusação tráfico ...,entretenimento,11/01/2021,\nEdgard Matsuki,https://www.boatos.org/religiao/papa-francisco...,0.0
1,Equador prepara cova coletiva para mortos por ...,NaN,o governar equador anunciar preparar cova cole...,saúde,27/03/2020 18h25,27/03/2020 18h25,https://noticias.uol.com.br/internacional/ulti...,1.0
2,Air France voltará a operar voo direto Pequim-...,NaN,o companhia air france operar voar direto pequ...,saúde,07/08/2020 13h42,07/08/2020 13h42,https://www.uol.com.br/nossa/noticias/afp/2020...,1.0
3,Marfrig intensifica venda de carne do Brasil a...,NaN,o marfrig global foods retomar vender carnar b...,saúde,27/04/2020 14h53,27/04/2020 14h53,https://economia.uol.com.br/noticias/reuters/2...,1.0
4,As parciais das eleições de 2014 alternaram ma...,NaN,o assunto voltar o compartilhar rede social ju...,entretenimento,31/07/2021,Gilmar Lopes,https://www.e-farsas.com/as-parciais-das-eleic...,0.0


#### Nulos
Verificar presença de linha com campo 'Noticia' nulo e remover se for o caso

In [77]:
alternative_df.isna().sum()

Titulo         31
Subtitulo    6323
Noticia         1
Categoria       1
Data          352
Autor          17
URL             1
Classe          1
dtype: int64

In [78]:
alternative_df[alternative_df['Noticia'].isna()]

,Titulo,Subtitulo,Noticia,Categoria,Data,Autor,URL,Classe
7337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
alternative_df.dropna(subset=['Noticia'], inplace=True)

#### Preparação do segundo dataset

Colunas de interesse são apenas 'notícia' (equivalente a 'lemma_str') e classe (equivalente a 'fake'). Note que, neste dataset, Classe 0.0 indica notícia Falsa e classe 1.0 indica notícia real! Portanto, é necessário adaptar o DataFrame.

In [81]:
alternative_df['lemmas_str'] = alternative_df['Noticia']
alternative_df['fake'] = (~alternative_df['Classe'].astype(bool)).astype(int) # inverter 1 e 0

# Dropar colunas agora redundantes
alternative_df = alternative_df.drop(['Noticia', 'Classe'], axis=1)

Amostrar 400 linhas para teste

In [83]:
n = 400
sample_df = alternative_df.sample(n=n, random_state=42)

#### Predição

Aplicar TF-IDF para vetorizar, salvar dados no S3 e efetuar predição

In [85]:
x_test_alt, y_test_alt = create_x_y_2(sample_df, tfidf)
save_to_s3_bucket_as_libsvm(x_test_alt, y_test_alt, 
                            prefix='test', filename='model_2_alt.libsvm', tipo='modelo')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [86]:
y_pred_alt = make_prediction(xgb_predictor_2, model_name='model_2_alt')

#### Métricas

In [87]:
print(classification_report(y_test_alt, y_pred_alt))

              precision    recall  f1-score   support

           0       1.00      0.01      0.02       205
           1       0.49      1.00      0.66       195

    accuracy                           0.49       400
   macro avg       0.74      0.50      0.34       400
weighted avg       0.75      0.49      0.33       400



#### Salvar predição

In [88]:
df_pred_alt = pd.DataFrame({'pred_alt': y_pred_alt})
df_pred_alt.to_csv(f's3://{BUCKET_MODEL}/test/pred_alt.csv', encoding='utf-8', index=False)

### Encerrar modelo
Após pegar métricas e dados de interesse, encerrar modelo para evitar cobranças.

In [89]:
xgb_predictor_2.delete_endpoint(delete_endpoint_config=True) 